<a href="https://colab.research.google.com/github/jeromecamilleri/reseauElectrique/blob/main/SimuElec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.widgets as widgets
import mplcursors
import ipywidgets as widgets
from IPython.display import display

def simulate_network_with_hover(N=50):
    plt.clf()
    G = nx.Graph()

    # Ajouter les noeuds
    for i in range(1, N + 1):
        G.add_node(i)

    # Connexions minimales pour éviter les noeuds isolés
    for i in range(1, N):
        G.add_edge(i, i + 1)

    # Connexions supplémentaires aléatoires (réseau réaliste mais non dense)
    extra_edges = int(N * 1.5)
    while G.number_of_edges() < extra_edges:
        u, v = np.random.choice(N, 2, replace=False) + 1
        if not G.has_edge(u, v):
            G.add_edge(u, v)

    # Définir producteurs et consommateurs
    nb_prod = max(1, N // 10)
    nb_cons = max(1, N * 3 // 10)

    producteurs = list(range(1, nb_prod + 1))
    consommateurs = list(range(nb_prod + 1, nb_prod + nb_cons + 1))

    # Intensités fictives dans les arêtes
    edge_currents = {}
    for u, v in G.edges():
        i_u = 100 if u in producteurs else (-50 if u in consommateurs else 0)
        i_v = 100 if v in producteurs else (-50 if v in consommateurs else 0)
        intensity = abs(i_u - i_v)
        edge_currents[(u, v)] = intensity
        G.edges[u, v]["intensity"] = intensity

    # Layout
    pos = nx.spring_layout(G, seed=42)

    # Couleurs des noeuds
    color_map = ['green' if n in producteurs else 'red' if n in consommateurs else 'gray' for n in G.nodes]

    # Affichage
    fig, ax = plt.subplots(figsize=(12, 8))

    # Dessin des arêtes avec largeur proportionnelle au courant
    widths = [0.5 + edge_currents[(u, v)] / 50 for u, v in G.edges()]
    lines = nx.draw_networkx_edges(G, pos, edge_color="blue", width=widths, ax=ax)

    # Dessin des noeuds
    nx.draw_networkx_nodes(G, pos, node_color=color_map, node_size=300, ax=ax)
    nx.draw_networkx_labels(G, pos, ax=ax)

    # Curseur survol des arêtes
    cursor = mplcursors.cursor(lines, hover=True)

    @cursor.connect("add")
    def on_add(sel):
        edge_index = sel.index
        u, v = list(G.edges())[edge_index]
        intensity = G.edges[u, v]["intensity"]
        sel.annotation.set(text=f"{u} ⟷ {v}\n{intensity} A")
        sel.annotation.get_bbox_patch().set(fc="yellow", alpha=0.8)

    ax.set_title(f"Réseau ({N} nœuds) — Producteurs en vert, consommateurs en rouge")
    ax.axis('off')
    plt.show()

# Interface utilisateur avec slider
slider = widgets.IntSlider(value=50, min=10, max=200, step=10, description='Nb Noeuds:')
ui = widgets.VBox([slider])
out = widgets.interactive_output(simulate_network_with_hover, {'N': slider})
display(ui, out)

Output()

In [13]:
!pip install mplcursors